In [1]:
import cv2 
from matplotlib import pyplot as plt

# 시작 함수

def stitching(dec, func):

    img1, img2, gray1, gray2 = loadImage('C:\\Users\\HwaJong\\Desktop\\test\\koreatech\\image1.jpg', \
                                        'C:\\Users\\HwaJong\\Desktop\\test\\koreatech\\image2.jpg')

    key1, key2, des1, des2 = findKeyPoints(gray1, gray2, dec)
    matching(img1, img2, key1, key2, des1, des2, func)

In [2]:
# 이미지 로드

def loadImage(path1, path2):
    img1 = cv2.imread(path1)
    img2 = cv2.imread(path2)

    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    return img1, img2, gray1, gray2

In [3]:
# detector 생성 및 특징점 추출

def findKeyPoints(img1, img2, name):

    # 추출기 생성
    if name == 'orb':
        obj = cv2.ORB_create()
    elif name == 'sift':
        obj = cv2.xfeatures2d.SIFT_create()
    elif name == 'surf':
        obj = cv2.xfeatures2d.SURF_create()
    elif name == 'kaze':
        obj = cv2.KAZE_create()
    elif name == 'brisk':
        obj = cv2.BRISK_create()
    elif name == 'akaze':
        obj = cv2.AKAZE.create()

    keypoints_1, descriptors_1 = obj.detectAndCompute(img1,None)
    keypoints_2, descriptors_2 = obj.detectAndCompute(img2,None)
    
    print('keypoints_1 :',len(keypoints_1), ' / descriptors_1:', descriptors_1.shape)
    print('keypoints_2 :',len(keypoints_2), ' / descriptors_2:', descriptors_2.shape)
    
    return keypoints_1, keypoints_2, descriptors_1, descriptors_2

In [4]:
# 두 개 이미지의 특징점 매칭

def matching(img1, img2, key1, key2, des1, des2, func):
    
    if func == 'match':
        #feature matching
        matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
        matches = matcher.match(des1,des2)
        
        # 매칭 결과를 거리기준 오름차순으로 정렬
        matches = sorted(matches, key = lambda x:x.distance)
        img3 = cv2.drawMatches(img1, key1, img2, key2, matches[:600], img2, flags=2)
        
        
    elif func == 'knn':
        matcher = cv2.BFMatcher()
        matches = matcher.knnMatch(des1, des2, 2)
        good = [[first] for first, second in matches if first.distance < 0.75 * second.distance]
        img3 = cv2.drawMatchesKnn(img1, key1, img2, key2, good, None, flags=2)

    plt.imshow(img3)
    plt.show()

In [ ]:
# detector 종류
detector = ['orb', 'sift', 'surf', 'kaze', 'akaze', 'brisk']

# match() 사용
for i in detector:
    print(i, 'match')
    stitching(i, 'match')
    
# knnMatch() 사용
for i in detector:
    print(i, 'knn')
    stitching(i, 'knn')